**Compile wrf chem in Parallella Parallella single board computer (PL): QUICK NOTE**

1. It is based on the NCAR wrf chem compile complete process and note on wrf chem compile by [this](http://sergioibarra.blogspot.in/2012/10/wrf-chem-compilacion-en-ubuntu-1204.html)
1. Loaded a ubuntu trusty based image from [here](http://www.parallella.org/create-sdcard/). Solved the issue of no Internet in PL by editing the interfaces file as follows and changing the files networkmanager.onf and remove net rules as stated in Parallella up and run note.
```bash
# interfaces(5) file used by ifup(8) and ifdown(8)
# Include files from /etc/network/interfaces.d:
source-directory /etc/network/interfaces.d
auto lo eth0
iface lo inet loopback
auto eth0
iface eth0 inet static
        address 192.168.1.111
        netmask 255.255.255.0
        network 192.168.1.0
        broadcast 192.168.1.255
        gateway 192.168.1.1
        dns-nameservers 192.168.1.1 8.8.8.8
#auto eth0
#iface eth0 inet dhcp
#auto wlan0
#iface wlan0 inet dhcp
```
1. The ubunut trusty image was loaded into a 32 GB SD card stored along with all requiered wrf files such as geog file, emission files. The SD card was mounted by command ```sudo mount /dev/mmcblk0p1 /home/wrf```
1. Then installed install basic required libs
```bash                           
sudo apt-get install g++
sudo apt-get install gfortran
sudo apt-get install gcc
```
1. Basic environment test, make sure the test passes all good for libs checks such as, gcc, gfortran, gcc+gfortran, perl, csh.
1. Made sure the gcc and gfortran versions are matching, current case version is 4.8.2 for both gcc and gfortran. 
1. To get the required libraries and other program source code run following command in the console
```bash			
wget http://www2.mmm.ucar.edu/wrf/src/WRFV3.4.1.TAR.gz
wget http://www2.mmm.ucar.edu/wrf/src/WPSV3.4.1.TAR.gz
wget http://www2.mmm.ucar.edu/wrf/src/WRFV3-Chem-3.4.1.TAR.gz
wget http://www2.mmm.ucar.edu/wrf/OnLineTutorial/compile_tutorial/tar_files/mpich-3.0.4.tar.gz
wget ftp://ftp.unidata.ucar.edu/pub/netcdf/netcdf-4.1.1.tar.gz
```

**Basic library compile MPICH and NETCDF**

1. For setting up PATH for NETCDF v4.1.1, given follwoing commands
```bash
tar xzvf netcdf-4.1.1.tar.gz
cd netcdf-4.1.1
export DIR=/home/linaro/wrf/WRFCHEMA1/libs
export CC=gcc
export CXX=g++
export FC=gfortran
export FCFLAGS=-g
export F77=gfortran
export FFLAGS=-g
export CXXFLAGS=-g
./configure --prefix=$DIR/netcdf --disable-dap \
--disable-netcdf-4 --disable-shared
make 
(this command ends in error related to documentation, there is a patch for this error) but ends in error of ```/bin/bash: texi2dvi: command not found```, oc by  ```sudo apt-get install texinfo```, ends in  another error need to install ``` sudo apt-get install texlive```, ends in another install, have to install ```sudo apt-get install groff```
make install
export PATH=$DIR/netcdf/bin:$PATH
export NETCDF=$DIR/netcdf
```

**setting up PATH for mpich and compiling**

1. First untar the mpich file

```bash
tar xzvf mpich-3.0.4.tar.gz     
cd mpich-3.0.4
./configure --prefix=$DIR/mpich
make
make install
export PATH=$DIR/mpich/bin:$PATH
```

1. Remaining required libraries were installed by normal sudo apt-get command, libs required were of 
```bash        
sudo apt-get install m4
sudo apt-get install libjasper-dev
sudo apt-get install libpng-dev
sudo apt-get install libnetcdf-dev
sudo apt-get install zlib1g-dev
sudo apt-get install build-essential
sudo apt-get install libcloog-ppl-dev
sudo apt-get install tcsh
sudo apt-get install libcloog-ppl0
```
1. BasicBasic environment test, make sure the test passes all good for libs checks such as, gcc, gfortran, gcc+gfortran, perl, csh. In much of the test the architeture mentions as -m64 was replaced with -g to run the test and all test was passed good. Similarly library check was undergone, which checks netcdf and mpich , this tests also passed good.

**Compiling WRF**

1. Edited the file sudo nano /etc/bash.bashrc, added following lines in it
```bash
export NETCDF=/home/linaro/wrf/WRFCHEMA1/libs/netcdf
export JASPERLIB=/usr/lib
export JASPERINC=/usr/include
export WRF_EM_CORE=1
export WRF_NMM_CORE=0
export WRF_CHEM=1
export WRF_KPP=0
export WRFIO_NCD_LARGE_FILE_SUPPORT=1
```
1. Based on [this](http://supersmith.com/site/ARM_files/wrf_on_arm.pdf) work of porting WRF for ARM, it is found that file ```configure_new.defaults``` in arch folder of source has to be edited to replace word x86_64 with armv7l for gfortran compiler section. Note the word x86_64 has to be replaced only along with gfortran, other wise it will be problematic.
1. Inside the wrf chem folder in whcih the chem folder already copy pasted, run following commands
```bash				
cd /home/hoopoe/wrfchem341/WRFV3_mpichA4/WRFV3/
./configure
```
1. The configure.wrf file has to be edited since the groftran and gcc used is of 4.8.2 version. The lines starting with has to be as below
```bash
FORMAT_FIXED    =       -ffixed-form -cpp
FORMAT_FREE     =       -ffree-form -ffree-line-length-none -cpp
```
1. Then execute the command, ./compile em_real and ./compile emi_conv. This completes WRF CHEM compilation
1. For compiling WPS, unzip the wps folder and run the command  ```./configure``` and choose serial based wps, it ends in creation of file configure.wps. 
1. Edit the file configure.wps and change the line starting with exactly as below
```bash
#### Architecture specific settings ####
COMPRESSION_LIBS    = -L/usr/lib -ljasper -lpng -lz
COMPRESSION_INC     = -I/usr/include
#
#   Settings for Linux x86_64, gfortran    (serial) 
#
#
COMPRESSION_LIBS    = -L/usr/lib -ljasper -lpng -lz
COMPRESSION_INC     = -I/usr/include
```
1. Change the line in wps due to gfortran version used 4.8.2 as, ```CPP                 = /usr/bin/cpp -P -traditional```
1. Check the location mentioned in configure.wps for configure WRFV3. 
1. Then execute the command, ```./compile```, and it will generate the exe required for WPS. 
```bash
Saturday, 03. January 2015 11:05AM 
Attempt 1, didn't genrated any executables, took following time for to compile WRF CHEM
build started:   Fri Jan  2 03:06:23 UTC 2015
build completed: Fri Jan 2 12:57:25 UTC 2015
```

**compiling PREP_CHEM_SRC**

1. Faced lots of error in compiling and running this program, the program requires netcdf version 4.1.1 and HDF1.8.4, zlib 1.2.7, other versions will generate error. More over the lib hdf5 has to be compiled in the folder /usr/local/. After compiling, it has to run command ```ldconfig``` to make the links correct for the compiled exe. It can be checked by the command ```lddd prep_chem_sources_RADM_WRF_FIM.exe```, if any dependent library is broken it will show as not found. 
1. The source file for program and package can be downloaded by following commands
```bash
wget http://zlib.net/fossils/zlib-1.2.7.tar.gz
wget http://www.hdfgroup.org/ftp/HDF5/current/src/hdf5-1.8.14.tar
ftp://aftp.fsl.noaa.gov/divisions/taq/global_emissions/prep_chem_sources_v1.4_08aug2013.tar.gz
```
1. compiling zlib1.7.2, inside the source directory, given following command,
```bash
FC=gfortran CC=gcc ./configure --prefix=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/zlib
make
make install
```
1. compiling hdf5-1.8.4, inside the source directory, given following command,
```bash			
./configure --prefix=/usr/local --enable-shared --enable-hl --with-zlib=/home/ubuntu/wrf/libs/zlib --enable-fortran
make
sudo checkinstall
```
1. Then editing the prep_chme_src configure file, ```include.mk.opt.gfortran```, with lines
```bash
NETCDF=/home/hoopoe/wrfchem341/WRFV3_mpichA3/libs/netcdf
HDF5=/usr/local
HDF5_LIB=-L$(HDF5)/lib -lhdf5hl_fortran -lhdf5_fortran -lhdf5_hl -lhdf5 -L/home/ubuntu/wrf/libs/zlib/lib/ -lz
```
1. Made sure that there is a line containing wording for big endian memory as ```F_OPTS=  -Xpreprocessor -D$(CHEM) -O2 -fconvert=big-endian -frecord-marker=4```
1. Then saved the configure file and give the command
```bash
cd /home/hoopoe/wrfchem341/WRFV3_mpichA3/PREP-CHEM-SRC-1.4/bin/build/
make OPT=gfortran.wrf CHEM=RADM_WRF_FIM
```
1. This will generate the exe and check the exe using the command, ```ldd prep_chem_sources_RADM_WRF_FIM.exe```

**Running WRFCHEM in PL**

1. To get the geog data necessary for geogrid.exe, it is huge file of size nearing 16 GB, used the script of wrfems, ```ems_install.pl```, for get the geog folder to go as like the currently used geog folder. The c3.large of aws instance is providing 300 GB of SSD which is useful for add this much size files folder. The SSD is mounted in /mnt and the perl script was transfered to the folder aftre giving permission on the folder /mnt by ```sudo chmod -R ugo+rw /mnt``` based on [this](http://stackoverflow.com/questions/11917786/change-read-write-access-of-a-folder-in-ubuntu-linux), which give all the permission for any user on the system to copy or edit files in the /mnt like system folder. The script transfer operation was made using FILEZILLA and connected following [this](http://stackoverflow.com/questions/16744863/connect-to-amazon-ec2-file-directory-using-filezilla-and-sftp). After this the script was made into executables by ```cd /mnt; chmod a+x ems_install.pl``` and the script was run by the command ```perl ems_install.pl --install```. While the script execution it asks for location to download, specified /mnt and the files are saving in this folder, the geog folder location is as follows, ```/mnt/wrfems/data/geog```. 
1. To get the GLOBAL emission files necessary for ```prep_chem_src.exe``` is downloaded by this command
```bash
wget ftp://aftp.fsl.noaa.gov/divisions/taq/global_emissions/global_emissions_v3_02aug2012.tar.gz
tar xzvf global_emissions_v3_02aug2012.tar.gz
```
1. The global_emissions folder is not located files as per the requirement of prep_chem_src, following copy past  has to be made to avoid the error of files not found error. 
```bash
cp /mnt/Global_emissions_v3/Emission_data/GOCART/emissions/OC_anthro_noship_2006.nc /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp /mnt/Global_emissions_v3/Emission_data/GOCART/emissions/OC_anthro_noship_2006.nc /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp /mnt/Global_emissions_v3/Emission_data/GOCART/emissions/BC_anthro_noship_2006.nc /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp /mnt/Global_emissions_v3/Emission_data/GOCART/emissions/SO2_anthro_noship_2006.nc /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp /mnt/Global_emissions_v3/Emission_data/GOCART/dms_data/dms_1x1.25.nc /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp -r /mnt/Global_emissions_v3/Emission_data/GOCART/erod /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp -r /mnt/Global_emissions_v3/Emission_data/GOCART/dms_data /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cd /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
mkdir gocart_bg
cp -r * /mnt/Global_emissions_v3/Emission_data/GOCART/gocart_bg/
cp gmi* gocart_bg/
```
1. The folder is having 5 GB size and it also must kept in the folder /mnt.
1. Based on the above folder location the variables has to be edited for namelist.wps and prep_chem_sources.inp.
1. A keep in mind fact is the aws volume here 8GB magnetic volume has to be in same zone to be detached and attached with another aws instances. 